# Cheapest flow of a given size/maximum


The Minimum Cost Flow (MCF) Algorithm is a method used in network optimization to find the cheapest way to send a specified amount of flow through a directed graph while respecting capacity constraints on edges.

The Minimum Cost Flow Algorithm is widely used in real-world applications, such as:

- Logistics & Transportation – Finding the cheapest way to deliver goods.

- Supply Chain Management – Distributing resources at minimal cost.

In [58]:
class CheapestFlow:
    def __init__(self, graph):
        self.graph = graph.copy()
        newnode = 0
        for u,v in graph:
            if (v,u) in self.graph:
                x = f"x{newnode}"
                self.graph[(u,x)] = self.graph[(u,v)].copy()
                self.graph[(x,v)] = self.graph[(u,v)].copy()
                self.graph[(x,v)]['cost'] = 0
                del self.graph[(u,v)]
                newnode+=1
        self.edges = list(self.graph.keys())
        # Inicjalizacja resztowych pojemności i kosztów dla każdej krawędzi
        self.residual_capacity = {edge: self.graph[edge]['capacity'] for edge in self.edges}
        self.cost = {edge: self.graph[edge]['cost'] for edge in self.edges}

        # Tworzenie odwrotnych krawędzi w grafie resztowym
        
        for u, v in self.edges:
            if (v, u) not in self.graph:
                self.residual_capacity[(v, u)] = 0  # Początkowa pojemność odwrotnej krawędzi wynosi 0
                self.cost[(v, u)] = -self.cost[(u, v)]  # Koszt odwrotnej krawędzi jest ujemny względem oryginalnej


    def bellman_ford(self, source, sink):
        # Inicjalizacja odległości od źródła do wszystkich węzłów jako nieskończoność
        nodes = set(node for edge in self.graph for node in edge)
        distance = {node: float('inf') for node in nodes}
        parent = {}  # Przechowuje ścieżkę do węzłów
        distance[source] = 0  # Odległość do źródła wynosi 0

        # Relaksacja krawędzi n-1 razy
        for _ in range(len(nodes) - 1):
            for u, v in self.residual_capacity.keys():
                # Jeśli istnieje dodatnia resztowa pojemność i koszt ścieżki jest mniejszy, aktualizujemy odległość
                if (self.residual_capacity[(u, v)] > 0 and 
                    distance[u] + self.cost[(u, v)] < distance[v]):
                    distance[v] = distance[u] + self.cost[(u, v)]
                    parent[v] = u  # Ustawiamy u jako rodzica v

        return distance[sink] < float('inf'), parent

    def fixed_flow_min_cost(self, source, sink, fixed_flow):
        min_cost = 0
        flow_sent = 0

        while flow_sent < fixed_flow:
            # Znajdowanie najkrótszej ścieżki w grafie resztowym za pomocą Bellmana-Forda
            path_found, parent = self.bellman_ford(source, sink)
            if not path_found:
                raise ValueError("No feasible flow of the required size exists.")

            # Znalezienie minimalnej pojemności na ścieżce (wąskiego gardła)
            path_flow = float('inf')
            v = sink
            while v != source:
                u = parent[v]
                path_flow = min(path_flow, self.residual_capacity[(u, v)])
                v = u

            # Ograniczenie przepływu do wymaganej wartości
            path_flow = min(path_flow, fixed_flow - flow_sent)
            
            # Aktualizacja resztowych pojemności i kosztu
            v = sink
            while v != source:
                u = parent[v]
                self.residual_capacity[(u, v)] -= path_flow  # Odejmujemy przesłany przepływ
                self.residual_capacity[(v, u)] += path_flow  # Dodajemy przepływ do odwrotnej krawędzi
                min_cost += path_flow * self.cost[(u, v)]  # Aktualizacja minimalnego kosztu
                v = u

            flow_sent += path_flow  # Aktualizacja całkowitego przesłanego przepływu

        return min_cost, self.residual_capacity

    def max_flow_min_cost(self, source, sink):
        max_flow = 0
        min_cost = 0

        while True:
            # Znajdowanie najkrótszej ścieżki w grafie resztowym za pomocą Bellmana-Forda
            path_found, parent = self.bellman_ford(source, sink)
            if not path_found:
                break

            # Znalezienie minimalnej pojemności na ścieżce (wąskiego gardła)
            path_flow = float('inf')
            v = sink
            while v != source:
                u = parent[v]
                path_flow = min(path_flow, self.residual_capacity[(u, v)])
                v = u

            # Aktualizacja resztowych pojemności i kosztu
            v = sink
            while v != source:
                u = parent[v]
                self.residual_capacity[(u, v)] -= path_flow  # Odejmujemy przepływ
                self.residual_capacity[(v, u)] += path_flow  # Dodajemy przepływ do odwrotnej krawędzi
                min_cost += path_flow * self.cost[(u, v)]  # Aktualizacja kosztu
                v = u

            max_flow += path_flow  # Zwiększenie maksymalnego przepływu

        return max_flow, min_cost


# Przykładowe użycie
if __name__ == "__main__":
    graph = {
        (0, 1): {'capacity': 6, 'cost': 5},
        (0, 2): {'capacity': 5, 'cost': 3},
        (1, 2): {'capacity': 4, 'cost': 3},
        (1, 3): {'capacity': 2, 'cost': 5},
        (2, 3): {'capacity': 3, 'cost': 6},
        (2, 4): {'capacity': 5, 'cost': 4},
        (3, 4): {'capacity': 3, 'cost': 2},
    }

    source = 0
    sink = 4
    flow = "max"
    min_cost_flow = CheapestFlow(graph)

    try:
        if flow != "max":
            min_cost, flow_dict = min_cost_flow.fixed_flow_min_cost(source, sink, flow)
            print("Minimum Cost to send fixed flow:", min_cost)
        else:
            max_flow, min_cost = min_cost_flow.max_flow_min_cost(source, sink)
            print("Maximum Flow:", max_flow)
            print("Minimum Cost of Maximum Flow:", min_cost)
    except ValueError as e:
        print(e)


{(0, 1): {'capacity': 6, 'cost': 5}, (0, 2): {'capacity': 5, 'cost': 3}, (1, 2): {'capacity': 4, 'cost': 3}, (1, 3): {'capacity': 2, 'cost': 5}, (2, 3): {'capacity': 3, 'cost': 6}, (2, 4): {'capacity': 5, 'cost': 4}, (3, 4): {'capacity': 3, 'cost': 2}}
[(0, 1), (0, 2), (1, 2), (1, 3), (2, 3), (2, 4), (3, 4)]
{(0, 1): 6, (0, 2): 5, (1, 2): 4, (1, 3): 2, (2, 3): 3, (2, 4): 5, (3, 4): 3, (1, 0): 0, (2, 0): 0, (2, 1): 0, (3, 1): 0, (3, 2): 0, (4, 2): 0, (4, 3): 0}
Maximum Flow: 8
Minimum Cost of Maximum Flow: 75


In [59]:
# Example Usage
if __name__ == "__main__":
    # Define the graph as a dictionary of edges
    graph = {
        ("s", "a"): {'capacity': 10, 'cost': 10},
        ("s", "b"): {'capacity': 5, 'cost': 1},
        ("a", "t"): {'capacity': 5, 'cost': 5},
        ("a", "b"): {'capacity': 5, 'cost': 5},
        ("b", "a"): {'capacity': 5, 'cost': 4},
        ("b", "t"): {'capacity': 10, 'cost': 10}
    }

    source = "s"
    sink = "t"
    flow = "max"  # Specify the required flow or set to 0 to calculate for maximum flow
    min_cost_flow = CheapestFlow(graph)

    try:
        if flow != "max":
            min_cost, flow_dict = min_cost_flow.fixed_flow_min_cost(source, sink, flow)
            print("Minimum Cost to send fixed flow:", min_cost)
        else:
            max_flow, min_cost = min_cost_flow.max_flow_min_cost(source, sink)
            print("Maximum Flow:", max_flow)
            print("Minimum Cost of Maximum Flow:", min_cost)

    except ValueError as e:
        print(e)

{('s', 'a'): {'capacity': 10, 'cost': 10}, ('s', 'b'): {'capacity': 5, 'cost': 1}, ('a', 't'): {'capacity': 5, 'cost': 5}, ('b', 'a'): {'capacity': 5, 'cost': 4}, ('b', 't'): {'capacity': 10, 'cost': 10}, ('a', 'x0'): {'capacity': 5, 'cost': 5}, ('x0', 'b'): {'capacity': 5, 'cost': 0}}
[('s', 'a'), ('s', 'b'), ('a', 't'), ('b', 'a'), ('b', 't'), ('a', 'x0'), ('x0', 'b')]
{('s', 'a'): 10, ('s', 'b'): 5, ('a', 't'): 5, ('b', 'a'): 5, ('b', 't'): 10, ('a', 'x0'): 5, ('x0', 'b'): 5, ('a', 's'): 0, ('b', 's'): 0, ('t', 'a'): 0, ('a', 'b'): 0, ('t', 'b'): 0, ('x0', 'a'): 0, ('b', 'x0'): 0}
Maximum Flow: 15
Minimum Cost of Maximum Flow: 255
